### 1. Instalación e inicialización de Spark

In [ ]:
# Actualización de los repositorios de UBUNTU
!sudo apt-get update

In [ ]:
# Instalación de JAVA
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Descarga de SPARK
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
# Des-zipeado del instalador
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
# Instalación de Spark en Python
!pip install -q findspark

In [ ]:
# Configuración de variables de entorno
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [ ]:
# Inicialización de Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
# Verificación de la sesión de Spark
spark

### 2. Creación de estructura de directorios para la capa UNIVERSAL

In [ ]:
# Librería para manipulación del sistema de archivos
import os

In [ ]:
# Importar librerias que vamos a utilizar
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import IntegerType, DoubleType, StringType
import pyspark.sql.functions as f

In [ ]:
# Vamos a modelar las tablas
#
# - ENTIDAD CLIENTE
# - ENTIDAD EMPRESA

In [ ]:
# Creamos el directorio que va almacenar los registros correctos (cumplen reglas de calidad) provenientes de los datos de CLIENTE
os.mkdir('/content/drive/MyDrive/DATALAKE/UNIVERSAL/CLIENTE')

In [ ]:
# Creamos el directorio que va almacenar los registros incorrectos (no cumplen reglas de calidad) provenientes de los datos de CLIENTE
os.mkdir('/content/drive/MyDrive/DATALAKE/UNIVERSAL/CLIENTE_REJ')

In [ ]:
# Creamos el directorio que va almacenar los registros correctos (cumplen reglas de calidad) provenientes de los datos de EMPRESA
os.mkdir('/content/drive/MyDrive/DATALAKE/UNIVERSAL/EMPRESA')

In [ ]:
# Creamos el directorio que va almacenar los registros incorrectos (no cumplen reglas de calidad) provenientes de los datos de EMPRESA
os.mkdir('/content/drive/MyDrive/DATALAKE/UNIVERSAL/EMPRESA_REJ')

### 3. Lectura, selección, casteo, limpieza, rejectados [CLIENTE]

In [ ]:
# [LECTURA]

In [ ]:
# Leemos la entidad "CLIENTE" casteando a los tipos de datos correctos
dfClienteLanding = spark.read.format("parquet").load("/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/CLIENTE/2018-04-19")

# Mostramos los datos
dfClienteLanding.show(truncate=False)

+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+
|ID |NOMBRE   |TELEFONO      |CORREO                                 |FECHA_INGRESO|EDAD|SALARIO|ID_EMPRESA|
+---+---------+--------------+---------------------------------------+-------------+----+-------+----------+
|1  |Carl     |1-745-633-9145|arcu.Sed.et@ante.co.uk                 |2004-04-23   |32  |20095  |5         |
|2  |Priscilla|155-2498      |Donec.egestas.Aliquam@volutpatnunc.edu |2019-02-17   |34  |9298   |2         |
|3  |Jocelyn  |1-204-956-8594|amet.diam@lobortis.co.uk               |2002-08-01   |27  |10853  |3         |
|4  |Aidan    |1-719-862-9385|euismod.et.commodo@nibhlaciniaorci.edu |2018-11-06   |29  |3387   |10        |
|5  |Leandra  |839-8044      |at@pretiumetrutrum.com                 |2002-10-10   |41  |22102  |1         |
|6  |Bert     |797-4453      |a.felis.ullamcorper@arcu.org           |2017-04-25   |70  |7800   |7         |
|7  |Mark     |1-68

In [ ]:
# [SELECCIÓN]

In [ ]:
# Seleccionamos los campos que el modelador nos indique para "CLIENTE" con el tipo de dato correcto
dfCliente1 = dfClienteLanding.select(
    dfClienteLanding["ID"].alias("ID_CLIENTE"),
    dfClienteLanding["NOMBRE"].alias("NOMBRE_CLIENTE"),
    dfClienteLanding["CORREO"].alias("CORREO_CLIENTE"),
    dfClienteLanding["EDAD"].alias("EDAD_CLIENTE"),
    dfClienteLanding["SALARIO"].alias("SALARIO_CLIENTE")                
)

# Mostramos los datos
dfCliente1.show(truncate=False)

+----------+--------------+---------------------------------------+------------+---------------+
|ID_CLIENTE|NOMBRE_CLIENTE|CORREO_CLIENTE                         |EDAD_CLIENTE|SALARIO_CLIENTE|
+----------+--------------+---------------------------------------+------------+---------------+
|1         |Carl          |arcu.Sed.et@ante.co.uk                 |32          |20095          |
|2         |Priscilla     |Donec.egestas.Aliquam@volutpatnunc.edu |34          |9298           |
|3         |Jocelyn       |amet.diam@lobortis.co.uk               |27          |10853          |
|4         |Aidan         |euismod.et.commodo@nibhlaciniaorci.edu |29          |3387           |
|5         |Leandra       |at@pretiumetrutrum.com                 |41          |22102          |
|6         |Bert          |a.felis.ullamcorper@arcu.org           |70          |7800           |
|7         |Mark          |Quisque.ac@placerat.ca                 |52          |8112           |
|8         |Jonah         |eu.

In [ ]:
# Mostramos el esquema de los datos
dfCliente1.printSchema()

root
 |-- ID_CLIENTE: string (nullable = true)
 |-- NOMBRE_CLIENTE: string (nullable = true)
 |-- CORREO_CLIENTE: string (nullable = true)
 |-- EDAD_CLIENTE: string (nullable = true)
 |-- SALARIO_CLIENTE: string (nullable = true)



In [ ]:
# [CASTEAR]

In [ ]:
# Casteamos los datos de los campos seleccionados
dfCliente2 = dfCliente1.\
             withColumn("ID_CLIENTE", dfCliente1["ID_CLIENTE"].cast(StringType())).\
             withColumn("NOMBRE_CLIENTE", dfCliente1["NOMBRE_CLIENTE"].cast(StringType())).\
             withColumn("CORREO_CLIENTE", dfCliente1["CORREO_CLIENTE"].cast(StringType())).\
             withColumn("EDAD_CLIENTE", dfCliente1["EDAD_CLIENTE"].cast(IntegerType())).\
             withColumn("SALARIO_CLIENTE", dfCliente1["SALARIO_CLIENTE"].cast(DoubleType()))

# Mostramos los datos
dfCliente2.show(truncate=False)

+----------+--------------+---------------------------------------+------------+---------------+
|ID_CLIENTE|NOMBRE_CLIENTE|CORREO_CLIENTE                         |EDAD_CLIENTE|SALARIO_CLIENTE|
+----------+--------------+---------------------------------------+------------+---------------+
|1         |Carl          |arcu.Sed.et@ante.co.uk                 |32          |20095.0        |
|2         |Priscilla     |Donec.egestas.Aliquam@volutpatnunc.edu |34          |9298.0         |
|3         |Jocelyn       |amet.diam@lobortis.co.uk               |27          |10853.0        |
|4         |Aidan         |euismod.et.commodo@nibhlaciniaorci.edu |29          |3387.0         |
|5         |Leandra       |at@pretiumetrutrum.com                 |41          |22102.0        |
|6         |Bert          |a.felis.ullamcorper@arcu.org           |70          |7800.0         |
|7         |Mark          |Quisque.ac@placerat.ca                 |52          |8112.0         |
|8         |Jonah         |eu.

In [ ]:
# Mostramos el esquema de los datos
dfCliente2.printSchema()

root
 |-- ID_CLIENTE: string (nullable = true)
 |-- NOMBRE_CLIENTE: string (nullable = true)
 |-- CORREO_CLIENTE: string (nullable = true)
 |-- EDAD_CLIENTE: integer (nullable = true)
 |-- SALARIO_CLIENTE: double (nullable = true)



In [ ]:
# [REGLAS DE CALIDAD (LIMPIEZA)]

In [ ]:
# Ahora viene la aplicación de las reglas de calidad, esto por supuesto ya es muy relativo va a depender de qué es lo que entienda como 
# reglas de calidad una empresa. 
# Datos de CLIENTE que cumplen las reglas de calidad
dfClienteUniversal = dfCliente2.filter(
    (dfCliente2["ID_CLIENTE"].isNotNull()) &
    (dfCliente2["NOMBRE_CLIENTE"].isNotNull()) &
    (dfCliente2["CORREO_CLIENTE"].isNotNull()) &
    (dfCliente2["EDAD_CLIENTE"].isNotNull()) &
    (dfCliente2["SALARIO_CLIENTE"].isNotNull()) &
    (dfCliente2["EDAD_CLIENTE"] > 0) &
    (dfCliente2["EDAD_CLIENTE"] < 70) &   
    (dfCliente2["SALARIO_CLIENTE"] >= 0)                         
)

# Mostramos los datos
dfClienteUniversal.show(truncate=False)

+----------+--------------+---------------------------------------------------------+------------+---------------+
|ID_CLIENTE|NOMBRE_CLIENTE|CORREO_CLIENTE                                           |EDAD_CLIENTE|SALARIO_CLIENTE|
+----------+--------------+---------------------------------------------------------+------------+---------------+
|1         |Carl          |arcu.Sed.et@ante.co.uk                                   |32          |20095.0        |
|2         |Priscilla     |Donec.egestas.Aliquam@volutpatnunc.edu                   |34          |9298.0         |
|3         |Jocelyn       |amet.diam@lobortis.co.uk                                 |27          |10853.0        |
|4         |Aidan         |euismod.et.commodo@nibhlaciniaorci.edu                   |29          |3387.0         |
|5         |Leandra       |at@pretiumetrutrum.com                                   |41          |22102.0        |
|7         |Mark          |Quisque.ac@placerat.ca                               

In [ ]:
# Datos de CLIENTE que NO cumplen las reglas de calidad
# Estos datos luego debiesen ser casteados al tipo STRING y posterior a eso escribirlos en el directorio.
dfClienteRejectados = dfCliente2.filter(
    ~(
        (dfCliente2["ID_CLIENTE"].isNotNull()) &
        (dfCliente2["NOMBRE_CLIENTE"].isNotNull()) &
        (dfCliente2["CORREO_CLIENTE"].isNotNull()) &
        (dfCliente2["EDAD_CLIENTE"].isNotNull()) &
        (dfCliente2["SALARIO_CLIENTE"].isNotNull()) &
        (dfCliente2["EDAD_CLIENTE"] > 0) &
        (dfCliente2["EDAD_CLIENTE"] < 70) &   
        (dfCliente2["SALARIO_CLIENTE"] >= 0)      
     )                   
)

# Mostramos los datos
dfClienteRejectados.show(truncate=False)

+----------+--------------+----------------------------+------------+---------------+
|ID_CLIENTE|NOMBRE_CLIENTE|CORREO_CLIENTE              |EDAD_CLIENTE|SALARIO_CLIENTE|
+----------+--------------+----------------------------+------------+---------------+
|6         |Bert          |a.felis.ullamcorper@arcu.org|70          |7800.0         |
|19        |Laura         |mollis@ornare.ca            |70          |17403.0        |
|63        |Sade          |In@utquam.com               |70          |11112.0        |
+----------+--------------+----------------------------+------------+---------------+



### 4. Lectura, selección, casteo, limpieza, rejectados [EMPRESA]

In [ ]:
# [LECTURA]

In [ ]:
# Leemos la entidad "EMPRESA" casteando a los tipos de datos correctos
dfEmpresaLanding = spark.read.format("parquet").load("/content/drive/MyDrive/DATALAKE/LANDING/SERVER_CLIENTES_EMPRESAS/EMPRESA/2018-04-19")

# Mostramos los datos
dfEmpresaLanding.show(truncate=False)

+---+---------+
|ID |NOMBRE   |
+---+---------+
|1  |Walmart  |
|2  |Microsoft|
|3  |Apple    |
|4  |Toyota   |
|5  |Amazon   |
|6  |Google   |
|7  |Samsung  |
|8  |HP       |
|9  |IBM      |
|10 |Sony     |
+---+---------+



In [ ]:
# [SELECCIÓN]

In [ ]:
# Seleccionamos los campos que el modelador nos indique para "EMPRESA" con el tipo de dato correcto
dfEmpresa1 = dfEmpresaLanding.select(
    dfEmpresaLanding["ID"].alias("ID_EMPRESA"),
    dfEmpresaLanding["NOMBRE"].alias("NOMBRE_EMPRESA")              
)

# Mostramos los datos
dfEmpresa1.show(truncate=False)

+----------+--------------+
|ID_EMPRESA|NOMBRE_EMPRESA|
+----------+--------------+
|1         |Walmart       |
|2         |Microsoft     |
|3         |Apple         |
|4         |Toyota        |
|5         |Amazon        |
|6         |Google        |
|7         |Samsung       |
|8         |HP            |
|9         |IBM           |
|10        |Sony          |
+----------+--------------+



In [ ]:
# Mostramos el esquema de los datos
dfEmpresa1.printSchema()

root
 |-- ID: string (nullable = true)
 |-- NOMBRE: string (nullable = true)



In [ ]:
# [CASTEAR]

In [ ]:
# Casteamos los datos
dfEmpresa2 = dfEmpresa1.\
             withColumn("ID_EMPRESA", dfEmpresa1["ID_EMPRESA"].cast(StringType())).\
             withColumn("NOMBRE_EMPRESA", dfEmpresa1["NOMBRE_EMPRESA"].cast(StringType()))

# Mostramos los datos
dfEmpresa2.show(truncate=False)  

+----------+--------------+
|ID_EMPRESA|NOMBRE_EMPRESA|
+----------+--------------+
|1         |Walmart       |
|2         |Microsoft     |
|3         |Apple         |
|4         |Toyota        |
|5         |Amazon        |
|6         |Google        |
|7         |Samsung       |
|8         |HP            |
|9         |IBM           |
|10        |Sony          |
+----------+--------------+



In [ ]:
# Mostramos el esquema de los datos
dfEmpresa2.printSchema()

root
 |-- ID_EMPRESA: string (nullable = true)
 |-- NOMBRE_EMPRESA: string (nullable = true)



In [ ]:
# [REGLAS DE CALIDAD (LIMPIEZA)]

In [ ]:
# Datos de EMPRESA que cumplen las reglas de calidad
dfEmpresaUniversal = dfEmpresa2.filter(
    (dfEmpresa2["ID_EMPRESA"].isNotNull()) &
    (dfEmpresa2["NOMBRE_EMPRESA"].isNotNull())                      
)

# Mostramos los datos
dfEmpresaUniversal.show(truncate=False)

+----------+--------------+
|ID_EMPRESA|NOMBRE_EMPRESA|
+----------+--------------+
|1         |Walmart       |
|2         |Microsoft     |
|3         |Apple         |
|4         |Toyota        |
|5         |Amazon        |
|6         |Google        |
|7         |Samsung       |
|8         |HP            |
|9         |IBM           |
|10        |Sony          |
+----------+--------------+



In [ ]:
# Datos de EMPRESA que NO cumplen las reglas de calidad
# Estos datos luego debiesen ser casteados al tipo STRING y posterior a eso escribirlos en el directorio.
dfEmpresaRejectados = dfEmpresa2.filter(
    ~(
        (dfEmpresa2["ID_EMPRESA"].isNotNull()) &
        (dfEmpresa2["NOMBRE_EMPRESA"].isNotNull())      
     )                   
)

# Mostramos los datos
dfEmpresaRejectados.show(truncate=False)

+----------+--------------+
|ID_EMPRESA|NOMBRE_EMPRESA|
+----------+--------------+
+----------+--------------+



### 5. Escribimos los datos modelados

In [ ]:
# Realizamos la escritura de los datos de "CLIENTE" modelados y lo guardamos en el directorio CLIENTE en la capa UNIVERSAL,
# más especificamente, en el subdirectorio de partición para la fecha '2018-04-19' 
dfClienteUniversal.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/UNIVERSAL/CLIENTE/2018-04-19")

In [ ]:
# Realizamos la escritura de los datos de "CLIENTE_REJ" y lo guardamos en el directorio CLIENTE_REJ en la capa UNIVERSAL,
# más especificamente, en el subdirectorio de partición para la fecha '2018-04-19' 
dfClienteRejectados.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/UNIVERSAL/CLIENTE_REJ/2018-04-19")

In [ ]:
# Realizamos la escritura de los datos de "EMPRESA" modelados y lo guardamos en el directorio EMPRESA en la capa UNIVERSAL,
# más especificamente, en el subdirectorio de partición para la fecha '2018-04-19' 
dfEmpresaUniversal.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/UNIVERSAL/EMPRESA/2018-04-19")

In [ ]:
# Realizamos la escritura de los datos de "EMPRESA_REJ" y lo guardamos en el directorio EMPRESA_REJ en la capa UNIVERSAL,
# más especificamente, en el subdirectorio de partición para la fecha '2018-04-19' 
dfEmpresaRejectados.write.format("parquet").mode("overwrite").save("/content/drive/MyDrive/DATALAKE/UNIVERSAL/EMPRESA_REJ/2018-04-19")